# To do
* Other problems:
    * Ben Watson
* Doesnt exist in college db:
    * Carson Wentz
    * Delanie Walker
    * Cooper Kupp
    * Goedert
    * Marquise Colston
    * Flacco
* Find people who dont show up in inner joins due to spelling

In [1]:
###################### Import Packages #############################################
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from ffb_functions import *
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

start_year = 2004
end_year = 2019

# College stats cleaning

In [2]:
# college data pull 1: college stats: school, conf, games, stats, year
college_0 = pd.read_csv('college_stats.csv')
college_0 = college_0.loc[(college_0.Year >= start_year - 4)
                       & (college_0.Year <= end_year + 1)].reset_index(drop = True)

## to help the join
college_0['School'] = college_0['School'].str.replace('Pitt', 'Pittsburgh')
college_0['School'] = college_0['School'].str.replace('Ole Miss', 'Mississippi')
college_0['School'] = college_0['School'].str.replace('State', 'St.')
college_0['School'] = college_0['School'].str.replace('UAB', 'Ala-Birmingham')
college_0['School'] = college_0['School'].str.replace('Western Michigan', 'West. Michigan')
college_0['School'] = college_0['School'].str.replace('UCF', 'Central Florida')
college_0['School'] = college_0['School'].str.replace('Texas Christian', 'TCU')
college_0['School'] = college_0['School'].str.replace('Boston College', 'Boston Col.')
college_0['School'] = college_0['School'].str.replace('Brigham Young', 'BYU')
college_0['Name'] = college_0['Name'].str.lower()
college_0['Name'] = college_0['Name'].str.replace(' jr.', '')
college_0['Name'] = college_0['Name'].str.replace('Benjamin', 'Ben')
college_0['Name'] = college_0['Name'].str.replace('Mitch', 'Mitchell')
college_0['Name'] = college_0['Name'].str.replace('William', 'Will')
college_0['Name'] = college_0['Name'].str.replace('E.J.', 'EJ')
college_0['Name'] = college_0['Name'].str.replace('D.J.', 'DJ')
#college_0 = college_0.replace('Benjamin Watson', 'Ben Watson')
#college_0 = college_0.replace('Mitch Trubisky', 'Mitchell Trubisky')
#college_0 = college_0.replace('Robert Griffin III', 'Robert Griffin')
#college_0 = college_0.replace('Odell Beckham Jr.', 'Odell Beckham')
#college_0 = college_0.replace('Ted Ginn Jr.', 'Ted Ginn')
#college_0 = college_0.replace('Devante Parker', 'DeVante Parker')
#college_0 = college_0.replace('E.J. Manuel', 'EJ Manuel')
#college_0 = college_0.replace('William Fuller', 'Will Fuller')
#college_0 = college_0.replace('Deandre Hopkins', 'DeAndre Hopkins')
college_0 = college_0.replace('Maurice Drew', 'Maurice Jones-Drew')
#college_0 = college_0.replace('Demarco Murray', 'DeMarco Murray')
#college_0 = college_0.replace('Desean Jackson', 'DeSean Jackson')
#college_0 = college_0.replace('Desean Jackson', 'DeSean Jackson')

# this is important because otherwise would exclude those without combine numbers
college_0 = college_0.rename(columns= {'School': 'College'})

In [3]:
# college data pull 2: draft pick: college, age, some stats
rk_0 = pd.read_csv('draft.csv')
rk_0 = rk_0.loc[(rk_0.Year >= start_year - 4)
                       & (rk_0.Year <= end_year + 1)].reset_index(drop = True)


rk_0['Name'] = rk_0['Name'].str.replace('DJ Chark', 'D.J. Chark')
rk_0.loc[rk_0.Name == 'Devin Funchess', 'FantPos'] = 'WR'
rk_0['Name'] = rk_0['Name'].str.lower()

In [71]:
# college data pull 3: combine numbers: school, height, weight, combine stats, position, name
comb_0 = pd.read_csv('combine.csv')
comb_0 = comb_0.loc[(comb_0.Year >= start_year - 4)
                       & (comb_0.Year <= end_year + 1)].reset_index(drop = True)

# two adrian petersons, only want oklahoma one
comb_0 = comb_0.loc[~((comb_0.Name == 'Adrian Peterson') 
                      & (comb_0.School == 'Georgia Southern'))].reset_index(drop = True)

comb_0 = comb_0.replace('Odell Beckham, Jr.', 'Odell Beckham')
comb_0 = comb_0.replace('E.J. Manuel', 'EJ Manuel')
comb_0['Name'] = comb_0['Name'].str.lower()

# Merge data

In [77]:
# merge rookie names, draft capital with combine statistics
draft_and_combine = rk_0.merge(comb_0, on = ['Name', 'FantPos', 'Year'], how = 'outer')
draft_and_combine = draft_and_combine.loc[draft_and_combine.Name != 'mike williams'].reset_index(drop = True)

## merge rookie names and draft capital with college statistics, keep inner joins for now
draft_combine_stats = draft_and_combine.merge(college_0.drop(columns = 'Year'), on = ['Name', 'College'], how = 'inner')
draft_combine_stats = draft_combine_stats.loc[draft_combine_stats.Name != 'mike williams'].reset_index(drop = True)
draft_combine_stats = draft_combine_stats.replace('d.k. metcalf', 'dk metcalf')

# rookie option makes the years line up properly
adp_frame_0 = pd.read_csv('adp.csv')
adp_frame_rk = adp_frame_0.loc[(adp_frame_0.Year >= start_year)
                       & (adp_frame_0.Year <= end_year + 1)].reset_index(drop = True)


adp_frame_rk['Name'] = adp_frame_rk['Name'].str.lower()
adp_frame_rk = adp_frame_rk.replace('NEP', 'NWE')
adp_frame_rk = adp_frame_rk.replace('RAM', 'STL')
adp_frame_rk = adp_frame_rk.replace('TBB', 'TAM')
# merge to adp for fantasy relevant players
test = draft_combine_stats.merge(adp_frame_rk[['Name', 'Year', 'Overall', 'Tm']], on = ['Name', 'Year', 'Tm'], how = 'inner')

try:
    adp_frame_alt_rk = pd.read_csv('adp_frame_alt_rk.csv')
except:
    adp_frame_alt_rk = ADP_assembly(2010, 2020, 'rookie')         # sometimes this link is a problem. to fix it, open up the link in a browser and run again
    adp_frame_alt_rk.to_csv('adp_frame_alt_rk.csv', index = False)
adp_frame_alt_rk['Name'] = adp_frame_alt_rk['Name'].str.lower()

# get std metric   
test_2 = test.merge(adp_frame_alt_rk, on = ['Name', 'Year'], how = 'left')


try:
    database = pd.read_csv('profootballfocus.csv')
except:
    database = data_assembly(2004, 2019)                            # can go back to at least 2000, limited by adp data
    database.to_csv('profootballfocus.csv', index = False)          # save original db
database['Name'] = database['Name'].str.lower()

## change columns to floats for metric creation
cols_to_change = ['VBD', 'PosRank', 'OvRank', 'Rk', 'Age'
                  , 'PPR', 'GS', 'G'
                  , 'PaTD', 'Cmp', 'Int', 'RuTD', 'ReTD'
                  , 'PaYds', 'RuYds', 'ReYds', 'Rec'
                  , 'RuY/A', 'ReYds/R', 'PaAtt', 'RuAtt', 'Tgt'
                  , '2PM', '2PP', 'Fmb', 'FL', 'TD.3']
for col in cols_to_change:
    database[col] = database[col].astype(float)


# get ppr next year
test_2 = test_2.rename(columns = {'TM': 'Tm'})
test_3 = test_2.merge(database[['Name', 'Year', 'PPR', 'Tm']], on = ['Name', 'Year', 'Tm'], how = 'left')
#test_3 = test_3.drop('Tm_x', axis = 1).rename(columns = {'Tm_y': 'Tm'})

# impute 0 for NaN points next year
test_3['PPR'] = test_3['PPR'].fillna(0)

# impute mean std for rookies
test_3['Std.Dev'] = test_3['Std.Dev'].fillna(test_3['Std.Dev'].mean())

# impute age 22 for missing
test_3['Age'] = test_3['Age'].fillna(22)

# impute 254 for undrafted
test_3['Pick'] = test_3['Pick'].fillna(254)

# impute combine values for those remaining
# having a problem with this
cols = ['Wt', 'height', 'Dash', 'Vertical', 'Bench', 'Broad_Jump', 'Three_Cone', 'Shuttle']
for i in cols:
    test_3[i] = test_3[i].astype(float)
    test_3[i] = test_3[i].fillna(test_3.groupby("FantPos")[i].transform('mean'))
    #test_3[i] = test_3[i].fillna(0)

    # join win pct
try:
    team_temp = pd.read_csv('team_frame.csv')
except:
    team_temp = team_assembly(2004, 2019)
    team_temp.to_csv('team_frame.csv', index = False)
    
# team temp is a year behind, bc rookie stuff is spring, team stuff tends to be fall?
team_temp['Year'] = team_temp['Year'] + 1

test_4 = test_3.merge(team_temp, on = ['Year', 'Tm'], how = 'outer')
#print(real_final.loc[real_final.Name == 'Julian Edelman'])
test_4.Win_PCT = test_4.Win_PCT.fillna(0.500)
#test_4.loc[test_4.Year == 2019]

# to model
test_4.to_csv('rookie_database.csv', index = False)

In [82]:
draft_combine_stats.loc[draft_combine_stats.Name == 'alex smith']
test_4.loc[test_4.Name == 'alex smith']
#test.groupby('Year').count()
#adp_frame_rk.loc[adp_frame_rk.Name == 'alex smith']
#database.rename(columns = {'Tm': 'TM'})
#database

,Pick,Tm,Name,Age,College,Year,FantPos,School,Ht,Wt,Dash,Vertical,Bench,Broad_Jump,Three_Cone,Shuttle,height,Conf,G,Cmp,PaAtt,Pct,PaYds,PaTD,Int,RuYds,RuTD,RuAtt,ReYds,ReTD,Rec,Overall,Std.Dev,PPR,Win_PCT
19,1.0,SFO,alex smith,21.0,Utah,2005,QB,Utah,6-4,217.0,4.71,32.000000,23.5,113.000000,6.820000,3.960000,76.0,MWC,12.0,214.0,317.0,67.5,2952.0,32.0,4.0,631.0,10.0,135.0,0.0,0.0,0.0,182.11,10.969149,27.3,12.5
34,71.0,TAM,alex smith,23.0,Stanford,2005,TE,Stanford,6-4,258.0,4.88,35.148649,28.0,119.526316,7.098235,4.335429,76.0,Pac-10,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,706.0,3.0,52.0,169.50,10.969149,89.7,31.2


In [81]:
rk_0.loc[rk_0.Name.str.contains('metcalf')]
#college_0.loc[college_0.Name.str.contains('metcalf')]
comb_0.loc[comb_0.FantPos.str.contains('QB')]
test_3.loc[test_3.FantPos.str.contains('QB')]

,Pick,Tm,Name,Age,College,Year,FantPos,School,Ht,Wt,Dash,Vertical,Bench,Broad_Jump,Three_Cone,Shuttle,height,Conf,G,Cmp,PaAtt,Pct,PaYds,PaTD,Int,RuYds,RuTD,RuAtt,ReYds,ReTD,Rec,Overall,Std.Dev,PPR
4,11.0,PIT,ben roethlisberger,22.0,Miami (OH),2004,QB,Miami (OH),6-5,241.0,4.750000,32.151515,23.5,115.0,7.036452,4.285312,77.0,MAC,14,342,495,69.1,4486,37,10,111,3,67,0,0,0,131.35,10.969149,171.2
16,90.0,ATL,matt schaub,23.0,Virginia,2004,QB,Virginia,6-6,243.0,5.040000,30.500000,23.5,110.0,7.650000,4.620000,78.0,ACC,11,281,403,69.7,2952,18,10,-23,1,26,0,0,0,247.97,10.969149,11.8
19,1.0,SFO,alex smith,21.0,Utah,2005,QB,Utah,6-4,217.0,4.710000,32.000000,23.5,113.0,6.820000,3.960000,76.0,MWC,12,214,317,67.5,2952,32,4,631,10,135,0,0,0,182.11,10.969149,27.3
35,106.0,CHI,kyle orton,22.0,Purdue,2005,QB,Purdue,6-4,233.0,5.060000,26.500000,23.5,115.0,7.380000,4.520000,76.0,Big Ten,11,236,389,60.7,3090,31,5,112,3,80,0,0,0,192.20,10.969149,89.2
39,3.0,TEN,vince young,23.0,Texas,2006,QB,Texas,6-5,229.0,4.480000,32.151515,23.5,115.0,7.036452,4.285312,77.0,Big 12,13,212,325,65.2,3036,26,10,1050,12,155,0,0,0,157.63,10.969149,209.2
41,10.0,ARI,matt leinart,23.0,USC,2006,QB,USC,6-5,223.0,4.900000,32.151515,23.5,115.0,7.036452,4.285312,77.0,Pac-10,13,283,431,65.7,3815,28,8,36,6,51,0,0,0,159.48,10.969149,138.8
42,11.0,DEN,jay cutler,23.0,Vanderbilt,2006,QB,Vanderbilt,6-3,226.0,4.770000,32.151515,23.0,115.0,7.100000,4.260000,75.0,SEC,11,273,462,59.1,3073,21,9,215,1,106,0,0,0,177.38,10.969149,67.8
61,1.0,OAK,jamarcus russell,22.0,LSU,2007,QB,LSU,6-5,265.0,4.830000,32.151515,23.5,115.0,7.036452,4.285312,77.0,SEC,13,232,342,67.8,3129,28,8,142,1,52,0,0,0,171.03,10.969149,17.3
65,22.0,CLE,brady quinn,22.0,Notre Dame,2007,QB,Notre Dame,6-4,232.0,4.730000,32.151515,24.0,115.0,7.036452,4.285312,76.0,Ind,13,289,467,61.9,3426,37,7,71,2,82,0,0,0,167.91,10.969149,1.8
83,3.0,ATL,matt ryan,23.0,Boston Col.,2008,QB,Boston Col.,6-5,228.0,4.890000,32.151515,23.5,115.0,7.400000,4.510000,77.0,ACC,14,388,654,59.3,4507,31,19,2,2,68,0,0,0,166.87,10.969149,198.0


# join team frame to this

In [9]:
team_frame = pd.read_csv('redraft_team_frame.csv')
team_frame

,Tm,Year,opp_difference,ru_opp
0,ARI,2004.0,0.0,-13.0
1,ARI,2005.0,-50.0,-360.0
2,ARI,2006.0,0.0,0.0
3,ARI,2007.0,88.0,0.0
4,ARI,2008.0,18.0,133.0
5,ARI,2009.0,127.0,-7.0
6,ARI,2010.0,65.0,141.0
7,ARI,2011.0,0.0,0.0
8,ARI,2012.0,-10.0,-69.0
9,ARI,2013.0,76.0,0.0
